In [1]:
from dotenv import load_dotenv

import os

### Set the Environment

In [2]:
load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")

### Call the Gemini-Pro

Vertex AI: https://ai.google.dev/gemini-api/docs/get-started/tutorial?lang=python

LangChain: https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [4]:
prompt = "Explain the quantum mechanic in one sentence"
result = llm.invoke(prompt)

In [5]:
print(result.content)

Quantum mechanics is a branch of science that studies the behavior of matter and energy at the atomic and subatomic levels, where classical mechanics fails to explain their behavior.


### Langchain Schema

In [42]:
# from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [49]:
messages = [
    SystemMessage(content="You are the Physicist and only response the output in Thai language only"),
    HumanMessage(content=prompt)
]

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro", convert_system_message_to_human=True)
result = llm.invoke(messages)

c:\Users\armlo\anaconda3\envs\langchain\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [50]:
result.content

'กลศาสตร์ควอนตัมอธิบายพฤติกรรมของสสารและพลังงานในระดับอะตอมและอนุภาคมูลฐาน'

### Caching LLM Response

In Memory Caching

In [63]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_google_genai import ChatGoogleGenerativeAI

set_llm_cache(InMemoryCache())

In [64]:
prompt = "Tell me a some dad jokes in two examples"

In [59]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
output = llm.invoke(prompt)
print(output.content)


1. What do you call a fish with no eyes? Fsh!
2. Why did the scarecrow win an award? Because he was outstanding in his field!
3. What do you call a belt made out of watches? A waist of time!
4. Why did the golfer wear two pairs of pants? In case he got a hole-in-one!
5. What do you call a lazy kangaroo? A pouch potato!
CPU times: total: 0 ns
Wall time: 2.24 s


In [60]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
output = llm.invoke(prompt)
print(output.content)


1. What do you call a fish with no eyes? Fsh!
2. Why did the scarecrow win an award? Because he was outstanding in his field!
3. What do you call a belt made out of watches? A waist of time!
4. Why did the golfer wear two pairs of pants? In case he got a hole-in-one!
5. What do you call a lazy kangaroo? A pouch potato!
CPU times: total: 0 ns
Wall time: 999 µs


SQLite Caching

In [65]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="./langchain.db"))

In [66]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt)

CPU times: total: 15.6 ms
Wall time: 1.5 s


AIMessage(content="1. What do you call a boomerang that doesn't come back? A stick!\n2. Why did the scarecrow win an award? Because he was outstanding in his field!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c82ab426-75c2-4398-b648-25c99ada289a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 39, 'total_tokens': 49})

In [67]:
%%time
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 14.2 ms


AIMessage(content="1. What do you call a boomerang that doesn't come back? A stick!\n2. Why did the scarecrow win an award? Because he was outstanding in his field!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c82ab426-75c2-4398-b648-25c99ada289a-0', usage_metadata={'input_tokens': 10, 'output_tokens': 39, 'total_tokens': 49})

### Langchain Streaming

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
prompt = "Tell me a chorus lyrics of 'talking to the moon' of Bruno mars"

In [13]:
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Talking to the moon
Trying to get to you
In hopes you're on the other side
Talking to me too
Or am I a fool
Who's trying to catch the moon

### Langchain PromptTemplates

In [15]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


In [22]:
template = """
    You are the expert virologist.
    Write a few sentence to explain about the {virus} in {language} language.
"""

prompt_template = PromptTemplate.from_template(template=template)

In [23]:
prompt = prompt_template.format(virus="HIV", language="Thai")
prompt

'\n    You are the expert virologist.\n    Write a few sentence to explain about the HIV in Thai language.\n'

In [24]:
llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
llm.invoke(prompt).content

'HIV (Human Immunodeficiency Virus) เป็นไวรัสที่ทำลายเซลล์เม็ดเลือดขาวชนิดหนึ่ง ทำให้ร่างกายอ่อนแอและติดเชื้อได้ง่าย หากไม่ได้รับการรักษา HIV อาจลุกลามไปเป็นโรคเอดส์ (Acquired Immunodeficiency Syndrome) ซึ่งเป็นระยะสุดท้ายของการติดเชื้อ HIV'

### Langchain ChatPromptTemplates

In [18]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage

llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro", convert_system_message_to_human=True)

In [42]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You have to response array in JSON format"),
        HumanMessagePromptTemplate.from_template("Top {n} countries at {continent} order by population")
    ]
)

messages = chat_template.format_messages(n=10, continent="asia")

In [43]:
output = llm.invoke(messages)

c:\Users\armlo\anaconda3\envs\langchain\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [44]:
print(output.content)

```json
[
  {
    "country": "China",
    "population": 1,444,244,680
  },
  {
    "country": "India",
    "population": 1,393,409,038
  },
  {
    "country": "Indonesia",
    "population": 273,523,615
  },
  {
    "country": "Pakistan",
    "population": 220,892,340
  },
  {
    "country": "Bangladesh",
    "population": 164,689,383
  },
  {
    "country": "Japan",
    "population": 126,440,000
  },
  {
    "country": "Philippines",
    "population": 109,581,078
  },
  {
    "country": "Vietnam",
    "population": 97,338,583
  },
  {
    "country": "Thailand",
    "population": 69,799,924
  },
  {
    "country": "Myanmar",
    "population": 54,409,794
  }
]
```


### Langchain Chained Messages

In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [47]:
template = """
    You are the expert virologist.
    Write a few sentence to explain about the {virus} in {language} language.
"""

prompt_template = PromptTemplate.from_template(template=template)

In [50]:
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True,
)

output = chain.invoke({"virus": "HIV", "language": "Thai"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are the expert virologist.
    Write a few sentence to explain about the HIV in Thai language.


> Finished chain.


In [51]:
output

{'virus': 'HIV',
 'language': 'Thai',
 'text': 'เชื้อ HIV เป็นเชื้อไวรัสที่ทำลายเซลล์เม็ดเลือดขาวชนิด CD4 ซึ่งเป็นเซลล์ที่สำคัญในระบบภูมิคุ้มกันของร่างกาย เมื่อร่างกายมีปริมาณเซลล์ CD4 ต่ำลง ระบบภูมิคุ้มกันก็จะอ่อนแอลง ทำให้เสี่ยงต่อการติดเชื้อฉวยโอกาสหรือโรคมะเร็งได้ง่ายขึ้น'}

### Langchain Squential Chains

In [54]:
# from langchain_google_genai import GoogleGenerativeAI
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain


# llm1 = GoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")
# llm2 = GoogleGenerativeAI(google_api_key=API_KEY, model="gemini-pro")

In [1]:
# prompt_template1 = PromptTemplate.from_template(
#     template = """You are the expert virologist.
#     Write a few sentence to explain about the {virus}"""
# )

# prompt_template2 = PromptTemplate.from_template(
#     template = """Convert the contents to {language}"""
# )

# chain1 = LLMChain(llm=llm1, prompt=prompt_template1.format(virus="HIV"))
# chain2 = LLMChain(llm=llm2, prompt=prompt_template2.format(language="Thai"))

In [2]:
# overall = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
# output = overall.invoke("virus")